In [1]:
import gc
import io
import os
import time

import numpy as np
import tensorflow as tf
import tensorflow_gan as tfgan
import logging
# Keep the import below for registering all model definitions
import sys
sys.path.append("../")
from models import ddpm
import losses
import sampling
from models import utils as mutils
from models.ema import ExponentialMovingAverage
import datasets
import evaluation
import likelihood
import sde_lib
from absl import flags
import torch
from torch.utils import tensorboard
from torchvision.utils import make_grid, save_image
from utils import save_checkpoint, restore_checkpoint
from configs.vp import cifar10_ddpm_continuous as configs
from sde_lib import VESDE, VPSDE, subVPSDE

/home/yifulu/.conda/envs/dpm/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

from dpm_solver import NoiseScheduleVP
config=configs.get_config()
config.eval.batch_size = 16
sde = VPSDE(beta_min=config.model.beta_min, beta_max=config.model.beta_max, N=config.model.num_scales)
sampling_eps = 1e-3
ns = NoiseScheduleVP('linear', continuous_beta_0=sde.beta_0, continuous_beta_1=sde.beta_1)
lst_steps = torch.tensor([1.0,0.950049877166748,0.9000999927520752,0.8501499891281128,
0.8001999258995056,0.750249981880188,0.7002999782562256,
0.6503499746322632,0.6003999710083008,0.5504499673843384,
0.5004999041557312,0.4505499303340912,0.40059995651245117,
0.35064995288848877,0.30069997906684875,0.25074997544288635,
0.20079998672008514,0.15084998309612274,0.10090000182390213,
0.050950001925230026,0.0010000000474974513,])

In [5]:
snrs = [1,1e-2]
lambdas = np.log(snrs)/2
print(ns.inverse_lambda(torch.tensor(lambdas)))

tensor([0.2590, 0.6760], dtype=torch.float64)
